In [1]:
#Razdel — сегментация текста на предложения и токены;
#Navec — качественный компактные эмбеддинги;
#Slovnet — современные компактные модели для морфологии, синтаксиса, NER;
#Yargy — правила и словари для извлечения структурированной информации;
#Ipymarkup — визуализация NER и синтаксической разметки;
!pip install natasha
!pip install natsort
!pip install navec

import os
import sys
import pickle
import re
from natsort import natsorted

import ipymarkup
from natasha import (
    Segmenter,
    MorphVocab,   
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger, 
    PER,
    NamesExtractor,
    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

In [2]:
def get_filepaths(d):
    # List which will store all of the full filepaths.
    locations = []
    # os.walk to get tree info
    for root, directories, files in os.walk(d):
        for filename in files:
            if filename.endswith(".pkl") and filename.startswith('ch'):
            # join the strings to form filepath
                filepath = os.path.join(root, filename)
                locations.append(filepath)
    # uses natsort to return human-intelligible ordering
    return natsorted(locations)

# store function results in a variable.   
sorted_filepaths = get_filepaths(r'C:\Users\emmak\dsam\bulgakov')
# print(sorted_filepaths)

In [3]:
from ipymarkup.span import Span
from ipymarkup.dep import Dep
from ipymarkup.demo import show_table

from ipymarkup import (
    show_span_box_markup,
    show_span_ascii_markup,
    show_dep_markup
)
from ipymarkup.palette import palette, BLUE, ORANGE, PURPLE, Rgb, Color, MaterialRgb


wholebook = []
# for every filepath in list of filepaths
for f in sorted_filepaths:
    # open filepath as a string
    with open(str(f), 'rb') as f:
        data = pickle.load(f)
        # joins the list from the pickle file
        ch = ' '.join(data)
        # appends list with joined chapter text
        wholebook.append(ch)
        f.close()

wholebook = ' '.join(wholebook)
doc = Doc(wholebook)
# divides doc into tokens and sents, given start and stop properties
doc.segment(segmenter)
# every token is morphologically tagged, given pos and feats properties
doc.tag_morph(morph_tagger)
# named entity recognition
doc.tag_ner(ner_tagger)
# extract names from PER span type
for span in doc.spans:
    # print(span.stop)
    # print(span.type)
    if span.type == PER:
        span.normalize(morph_vocab)
        span.extract_fact(names_extractor)

# print(wholebook)
# for f in wholebook:
#     doc = Doc(f)
#     # divides doc into tokens and sents, given start and stop properties
#     doc.segment(segmenter)
#     # every token is morphologically tagged, given pos and feats properties
#     doc.tag_morph(morph_tagger)
#     # named entity recognition
#     doc.tag_ner(ner_tagger)
#     extract names from PER span type
#     for span in doc.spans:
#         print(span.stop)
#         print(span.type)
#         if span.type == PER:
#             span.normalize(morph_vocab)
#             span.extract_fact(names_extractor)
    
    #doc.sents[0].morph.print()
    # print(doc.tokens[0])
    # #doc.sents.ner.print()
    # #print(doc.sents[0])
    # print(doc.spans[0])
    # print("\n")


In [4]:
dir(ipymarkup.palette)

['BLUE',
 'BROWN',
 'Color',
 'GREEN',
 'GREY',
 'MaterialRgb',
 'ORANGE',
 'PALETTE',
 'PURPLE',
 'Palette',
 'RED',
 'Record',
 'Rgb',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'material',
 'palette',
 'prepare_color',
 're']

In [5]:
TEAL = Color(
    'Teal',
    background=material('Teal', '50'),
    border=material('Teal', '100'),
    text=material('Teal', '300'),
    line=material('Teal', '200')
)

NameError: name 'material' is not defined

In [8]:
#Produces ipymarkup of the text, focusing only on spans under the PER category 
def getMarkup(doc):
    text  = doc.text
    # print(text)
    spans = []
    # types = {type for type in doc.spans}
    # print(doc.spans)
    for i, span in enumerate(doc.spans):
        span.normalize(morph_vocab)
        span.extract_fact(names_extractor)
        spans.append((span.start,span.stop,span.type))
    
    # print(spans)
    
    show_span_box_markup(text, spans, palette=palette({'PER':'blue','LOC':'purple', 'ORG':'#e0f2f1'}))
 
            
        # print(span.stop)
        # print(span.type)
    
    # if span.type == PER:
        #     span.normalize(morph_vocab)
        #     span.extract_fact(names_extractor)
    

In [9]:
getMarkup(doc)
# print(doc.spans[0])

KeyError: '#e0f2f1'